## Introduction
* For this notebook we will be exploring other stuctures to classify proteins from their sequence
* We will explore 4 different ways
* The first way, we will try changing some of the architecture using the emebedding and CONV1d
* The second way, we ill trying implemeting an LSTM after the embedding layer
* The third way, we will try implementing a GRU after the embedding layer
* The fourth way, we will try implementing bi bi-diertional LSTMS

## First Load in the data

In [2]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [3]:
import keras 
keras.__version__

'2.2.4'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
proteins = pd.read_csv("top_15_proteins.csv")

In [3]:
proteins.head()

,Unnamed: 0,sequence,classification,num_residues
0,67,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE,286
1,68,TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...,HYDROLASE,286
2,74,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,LIGASE,330
3,75,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,LIGASE,330
4,76,KESAAAKFERQHMDSGNSPSSSSNYCNLMMCCRKMTQGKCKPVNTF...,HYDROLASE,124


In [4]:
proteins.shape

(283101, 4)

### Tranform the labels

In [5]:
from sklearn.preprocessing import LabelBinarizer

# Transform labels to one-hot
lb = LabelBinarizer()
Y = lb.fit_transform(proteins['classification'])

In [6]:
lb.classes_

array(['HYDROLASE', 'HYDROLASE/HYDROLASE INHIBITOR', 'IMMUNE SYSTEM',
       'ISOMERASE', 'LIGASE', 'LYASE', 'OXIDOREDUCTASE', 'RIBOSOME',
       'RIBOSOME/ANTIBIOTIC', 'SIGNALING PROTEIN', 'TRANSCRIPTION',
       'TRANSFERASE', 'TRANSPORT PROTEIN', 'VIRAL PROTEIN', 'VIRUS'],
      dtype='<U29')

In [7]:
Y[34]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Vectorize the Sequences
* This time try adding padding pre

In [27]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# maximum length of sequence, everything afterwards is discarded!
max_length = 512

#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(proteins['sequence'])
#represent input data as word rank number sequences
X = tokenizer.texts_to_sequences(proteins['sequence'])
X = sequence.pad_sequences(X, maxlen=max_length,padding='pre')

In [28]:
tokenizer.word_docs

defaultdict(int,
            {'l': 271251,
             'f': 263330,
             'w': 218243,
             'e': 267895,
             'v': 269066,
             's': 268119,
             'n': 265583,
             'g': 276371,
             'q': 264838,
             't': 269758,
             'c': 213007,
             'd': 266266,
             'a': 277393,
             'r': 270110,
             'i': 267264,
             'y': 262657,
             'k': 270045,
             'p': 267751,
             'm': 258030,
             'h': 258236,
             'u': 5915,
             'x': 4954,
             'z': 3,
             'b': 4,
             'o': 2})

In [29]:
X[4545]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

### First Model
* Change embedding size 16
* Adding another conv1d and max pooling layer (with 16 filters and kernel size of 2)
* Now try 50 epochs this time

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

embedding_dim = 16

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(15, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Using TensorFlow backend.


NameError: name 'tokenizer' is not defined

In [36]:
X.shape,Y.shape

((283101, 512), (283101, 15))

### Split into train and test

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.1)

In [38]:
#keep X_test and y_test until the end get valids
X_test.shape
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.1)

In [39]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid), 
          epochs=50, 
          batch_size=128)

Train on 229311 samples, validate on 25479 samples
Epoch 1/50
229311/229311 [==============================] - 397s 2ms/step - loss: 1.3295 - acc: 0.5858 - val_loss: 0.9753 - val_acc: 0.7080
Epoch 2/50
229311/229311 [==============================] - 396s 2ms/step - loss: 0.8175 - acc: 0.7577 - val_loss: 0.7526 - val_acc: 0.7817
Epoch 3/50
229311/229311 [==============================] - 404s 2ms/step - loss: 0.6324 - acc: 0.8136 - val_loss: 0.6471 - val_acc: 0.8164
Epoch 4/50
229311/229311 [==============================] - 397s 2ms/step - loss: 0.5295 - acc: 0.8439 - val_loss: 0.5711 - val_acc: 0.8372
Epoch 5/50
229311/229311 [==============================] - 399s 2ms/step - loss: 0.4618 - acc: 0.8627 - val_loss: 0.5386 - val_acc: 0.8513
Epoch 6/50
229311/229311 [==============================] - 405s 2ms/step - loss: 0.4155 - acc: 0.8767 - val_loss: 0.5241 - val_acc: 0.8565
Epoch 7/50
229311/229311 [==============================] - 410s 2ms/step - loss: 0.3802 - acc: 0.8853 - val_

Save the the models and histories  to examine later

In [43]:
pd.DataFrame(model.history.history).to_csv('Protein_Classification_Parameter_Tuning_Other_Architectures/ThreeConvLayersHistory.csv')
                                           

In [44]:
model.save('Protein_Classification_Parameter_Tuning_Other_Architectures/ThreeConvLayers.h5')

In [45]:
del model

## Second  Model
* Use and LSTM layer after the embedding layer

In [56]:
from keras.layers import LSTM

embedding_dim = 16
depth  = 16

# create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(LSTM(depth))
model.add(Dense(15, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 512, 16)           416       
_________________________________________________________________
lstm_5 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_23 (Dense)             (None, 15)                255       
Total params: 2,783
Trainable params: 2,783
Non-trainable params: 0
_________________________________________________________________
None


Note this takes quite a long time for training. Use a smaller number of epochs

In [ ]:
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid), 
          epochs=15, 
          batch_size=128)

Train on 229311 samples, validate on 25479 samples
Epoch 1/15
229311/229311 [==============================] - 1646s 7ms/step - loss: 2.0589 - acc: 0.3382 - val_loss: 1.9320 - val_acc: 0.3604
Epoch 2/15
229311/229311 [==============================] - 1649s 7ms/step - loss: 1.9854 - acc: 0.3452 - val_loss: 1.9482 - val_acc: 0.3539
Epoch 3/15
229311/229311 [==============================] - 1625s 7ms/step - loss: 1.8543 - acc: 0.3877 - val_loss: 1.7856 - val_acc: 0.3983
Epoch 4/15
229311/229311 [==============================] - 1831s 8ms/step - loss: 1.7450 - acc: 0.4146 - val_loss: 1.6687 - val_acc: 0.4310
Epoch 5/15
229311/229311 [==============================] - 1656s 7ms/step - loss: 1.6560 - acc: 0.4444 - val_loss: 1.6301 - val_acc: 0.4451
Epoch 6/15
229311/229311 [==============================] - 1623s 7ms/step - loss: 1.6259 - acc: 0.4515 - val_loss: 1.5857 - val_acc: 0.4601
Epoch 7/15
229311/229311 [==============================] - 1675s 7ms/step - loss: 1.5930 - acc: 0.4626

In [55]:
del model